# SETUP

In [1]:
import sys
import json
from collections import Counter
import time
import re
import copy

# GLOBAL VARS

In [2]:
replay = "replay.json"
p1_me = list(range(1, 100))
p2_op = list(range(100, 200))

# HELPER FUNCTIONS

In [3]:
def assign_id(ids, card_name, card_state=None, obj_id=None, battle_position=None, face_up=None):
    first_id = ids.pop(0)
    return {"name": card_name, "id": first_id, "state": card_state, "object_id": obj_id, "battle_position": battle_position, "face_up": face_up}

def my_card(c, obj_id, all_cards):
    found = False
    for card in all_cards:
        if card["name"] == c and obj_id == card.get("object_id"):
            found = True
            break
    return found

def soft_my_card(c, all_cards):
    found = False
    for card in all_cards:
        if card["name"] == c:
            found = True
            break
    return found

def get_log(play):
    return play["log"].get("private_log") or play["log"].get("public_log")

# strip "(x/y)" trailing hints like "(4/5)" or "(1/1)"
_PAREN_HINT_RE = re.compile(r"\s*\(\s*\d+\s*/\s*\d+\s*\)\s*$")

def _strip_hint(s: str) -> str:
    return _PAREN_HINT_RE.sub("", s.strip())

def debug_state_play(state_bef, state_aft, play):
    print("\n=== DEBUG STATE PLAY ===")
    print("------------------ BEFORE:")
    print(json.dumps(state_bef, indent=2))
    print("------------------ AFTER:")
    print(json.dumps(state_aft, indent=2))
    print("------------------ PLAY:")
    print(json.dumps(play, indent=2))
    print("=======================\n")

def decompose_public_log(log: str):
    s = log.strip()

    # 1) Full: ... "<card>" from <start> to <finish>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+from\s+(?P<start>.+?)\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("start"))
        finish_location = _strip_hint(m.group("finish"))
        return action, card_name, start_location, finish_location

    # 2) Banished shortcut: ... banished "<card>" (x/y)? to <finish>  (no explicit 'from')
    m = re.match(
        r'^(?P<action>.+?)\s+banished\s+"(?P<card>[^"]+)"(?:\s*\(\s*\d+\s*/\s*\d+\s*\))?\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = "banished"
        finish_location = _strip_hint(m.group("finish"))
        return action, card_name, start_location, finish_location

    # 3) "in" form: ... "<card>" in <loc>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+in\s+(?P<loc>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("loc"))
        finish_location = None
        return action, card_name, start_location, finish_location

    # 4) "to"-only: ... "<card>" to <finish>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"(?:\s*\(\s*\d+\s*/\s*\d+\s*\))?\s+to\s+(?P<finish>.+)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = None
        finish_location = _strip_hint(m.group("finish"))
        return action, card_name, start_location, finish_location

    # 5) NEW: "from"-only: ... "<card>" from <start>
    m = re.match(
        r'^(?P<action>.+?)\s+"(?P<card>[^"]+)"\s+from\s+(?P<start>.+?)$',
        s, flags=re.IGNORECASE
    )
    if m:
        action = m.group("action").strip()
        card_name = m.group("card").strip()
        start_location = _strip_hint(m.group("start"))
        # Infer destination when action implies it
        finish_location = "banished" if "banish" in action.lower() else None
        return action, card_name, start_location, finish_location

    print("No match found.")
    return None

# GENERAL ANALYSIS

In [4]:
def parse(content):
    persistent_state = {
        # general
        "turn": 0,
        ### me
        "me_life": 8000,
        "me_hand": [],
        "me_GY": [],
        "me_banished": [],
        "me_field": {
            "M-1": None,
            "M-2": None,
            "M-3": None,
            "M-4": None,
            "M-5": None,
            "S-1": None,
            "S-2": None,
            "S-3": None,
            "S-4": None,
            "S-5": None,
        },
        "me_phase": None,
        ### opponent
        "op_life": 8000,
        "op_hand": [],
        "op_GY": [],
        "op_banished": [],
        "op_field": {
            "M-1": None,
            "M-2": None,
            "M-3": None,
            "M-4": None,
            "M-5": None,
            "S-1": None,
            "S-2": None,
            "S-3": None,
            "S-4": None,
            "S-5": None,
        },
        "op_phase": None,
    }
    p1 = content["player1"]["username"]
    p2 = content["player2"]["username"]

    for i, play in enumerate(content["plays"]):
        if play["play"] == "Admit defeat":
            print("Admit Defeat found, exiting parse.")
            break
        if play["play"] == "Life points":
            if play["username"] == p1:
                amount = play["amount"]
                persistent_state["me_life"] += amount
            else:
                amount = play["amount"]
                persistent_state["op_life"] += amount

        if play["play"] == "Pick first":
            cards = [(card["name"], card["object_id"]) for card in play["cards"]]
            cards_p1 = cards[:5]
            cards_p2 = cards[5:10]
            persistent_state["me_hand"] = [assign_id(p1_me, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p1]
            persistent_state["op_hand"] = [assign_id(p2_op, card, "private", obj_id, battle_position=None, face_up=None) for card, obj_id in cards_p2]

        if play["play"] == "Draw card":
            if play["username"] == p1:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["me_hand"].append(assign_id(p1_me, card_name, "private", obj_id, battle_position=None, face_up=None))
            else:
                card_name = play["card"]["name"]
                obj_id = play["card"]["object_id"]
                persistent_state["op_hand"].append(assign_id(p2_op, card_name, "private", obj_id, battle_position=None, face_up=None))

        if play["play"] == "Enter DP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "DP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "DP"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter SP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "SP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "SP"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter M1":
            if play["username"] == p1:
                persistent_state["me_phase"] = "M1"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "M1"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter BP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "BP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "BP"
                persistent_state["me_phase"] = None

        if play["play"] == "Enter M2":
            if play["username"] == p1:
                persistent_state["me_phase"] = "M2"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "M2"
                persistent_state["me_phase"] = None
                
        if play["play"] == "Enter EP":
            if play["username"] == p1:
                persistent_state["me_phase"] = "EP"
                persistent_state["op_phase"] = None
            else:
                persistent_state["op_phase"] = "EP"
                persistent_state["me_phase"] = None

        if play["play"] == "End turn":
            persistent_state["turn"] += 1
            if play["username"] == p1:
                persistent_state["me_phase"] = None
                persistent_state["op_phase"] = "DP"
            else:
                persistent_state["op_phase"] = None
                persistent_state["me_phase"] = "DP"

        if play["play"] == "Activate ST":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
    
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # update and move card
            card_entry["state"] = "public"
            card_entry["face_up"] = True
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            # remove from hand if necessary
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == "Set ST":
            card_object = play.get("card", None)
            card = card_object["name"] if card_object else None
            obj_id = card_object["object_id"] if card_object else None
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = soft_my_card(card_name, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"


            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card_name),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"
            
            # update and move card
            card_entry["state"] = "private"
            card_entry["face_up"] = False
            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            obj_id = card_entry.get("object_id", None)
            

            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]

        if play["play"] == "Normal Summon":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"

            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                raise ValueError("ACTIVATE ST - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            else:
                raise ValueError("ACTIVATE ST - unhandled start_location")
            
    
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            card_entry["state"] = "public"
            card_entry["face_up"] = True

            if finish_location:
                persistent_state[field_key][finish_location] = card_entry

            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
        
        if play["play"] == "To GY":
            card = play["card"]["name"]
            obj_id = play["card"]["object_id"]
            log = get_log(play)
            assert log is not None, f"No log found in play {i}"
            action, card_name, start_location, finish_location = decompose_public_log(log)
            assert card == card_name, f"Card name mismatch in play {i}: {card} vs {card_name}"
            all_me_cards = persistent_state["me_hand"] + persistent_state["me_GY"] + persistent_state["me_banished"] + [card for card in list(persistent_state["me_field"].values()) if card is not None]
            is_my_card = my_card(card, obj_id, all_me_cards)

            print(f"Action={action}, Card={card_name}, From={start_location}, To={finish_location}")
            print(is_my_card, play["username"], p1)

            if play["username"] == p1 and is_my_card:
                side = "me"
            elif play["username"] == p2 and not is_my_card:
                side = "op"
            else:
                if start_location != "Deck":                
                    raise ValueError("To GY - unexpected player/card ownership")
            
            # Programatic keys
            hand_key = f"{side}_hand"
            field_key = f"{side}_field"
            banish_key = f"{side}_banished"

            if start_location == "hand":
                source = persistent_state[hand_key]
            elif start_location in persistent_state[field_key]:
                # field is a dict of zones, flatten values for search
                source = [c for c in persistent_state[field_key].values() if c is not None]
            elif start_location == "banished":
                source = persistent_state[banish_key]
            elif start_location == "Deck":
                source = [assign_id(p1_me if side == "me" else p2_op, card, "public", obj_id, battle_position=None, face_up=None)]
            else:
                raise ValueError("To GY - unhandled start_location")
            
            card_entry = next(
                (c for c in source if c["name"] == card and c["object_id"] == obj_id),
                None
            )

            # WILL FAIL, BANISH NOT IMPLEMENTED
            assert card_entry is not None, f"Card {card} with obj_id {obj_id} not found in source in play {i}"

            # remove from source
            if start_location == "hand":
                persistent_state[hand_key] = [
                    c for c in persistent_state[hand_key] if c["object_id"] != obj_id
                ]
            elif start_location in persistent_state[field_key]:
                for zone, c in persistent_state[field_key].items():
                    if c is not None and c["object_id"] == obj_id:
                        persistent_state[field_key][zone] = None
                        break
            elif start_location == "banished":
                persistent_state[banish_key] = [
                    c for c in persistent_state[banish_key] if c["object_id"] != obj_id
                ]
            elif start_location == "Deck":
                pass  # already handled by creating a new entry


            # add to GY
            card_entry["state"] = "public"
            card_entry["face_up"] = None
            persistent_state[f"{side}_GY"].append(card_entry)
            


    print(json.dumps(persistent_state, indent=2))


parse(json.load(open(replay, 'r')))

Action=Sent, Card=Yellow Gadget, From=M-3, To=GY
True smashingyourfissure smashingyourfissure
Action=Sent, Card=Dimensional Prison, From=S-3, To=GY
True smashingyourfissure smashingyourfissure
Action=Sent, Card=Mystical Space Typhoon, From=S-3, To=GY
True smashingyourfissure smashingyourfissure
Action=Sent, Card=Caius the Shadow Monarch, From=Deck, To=GY
False smashingyourfissure smashingyourfissure
Action=Sent, Card=Green Gadget, From=banished, To=GY
True smashingyourfissure smashingyourfissure


AssertionError: Card Green Gadget with obj_id 3 not found in source in play 16